# 2022-2023 Interação em Linguagem Natural
# Grupo 1
# 59099 Duarte Raposo
# 49420 João Ferraz

In [ ]:
!pip install transformers datasets accelerate evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Base Model importing and testing

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, pipeline
base_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
config = RobertaConfig.from_pretrained(
    'roberta-base',
    num_labels=2,
)
base_model = RobertaForSequenceClassification.from_pretrained('roberta-base', config=config)

text = "Replace me by any text you'd like."
encoded_input = base_tokenizer(text, return_tensors='pt')
output = base_model(**encoded_input)
#output

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [ ]:
from datasets import load_dataset
dataset_full = load_dataset("super_glue", 'boolq')
dataset_train = dataset_full["train"]
dataset_val = dataset_full["validation"]
dataset_full

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})

# Experiments with tokenizer

In [ ]:
from tokenizers import trainers, ByteLevelBPETokenizer, pre_tokenizers
tokenizer_obj = ByteLevelBPETokenizer()
trainer = trainers.BpeTrainer()
tokenizer_obj.train_from_iterator(dataset_train["question"])
tokenizer_obj.train_from_iterator(dataset_train["passage"])

tokenizer = base_tokenizer
tokenizer.add_tokens(tokenizer_obj.get_vocab())
#tokenizer_obj.get_vocab()

1

In [ ]:
print(tokenizer.get_vocab)
print(base_tokenizer.get_vocab)

<bound method RobertaTokenizer.get_vocab of RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)>
<bound method RobertaTokenizer.get_vocab of RobertaTokenizer(name_or_path=

In [ ]:
tokenizer(dataset_train["question"][0])

{'input_ids': [0, 5016, 10209, 260, 8, 9724, 5158, 7566, 1994, 5, 276, 2777, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
print(tokenizer.convert_ids_to_tokens(0))
print(tokenizer.convert_ids_to_tokens(1))
print(tokenizer.convert_ids_to_tokens(2))

<s>
<pad>
</s>


# Dataset Preprocessing

In [ ]:
train_tokens = tokenizer(dataset_train["question"], dataset_train["passage"], truncation=False, padding=False)
val_tokens = tokenizer(dataset_val["question"], dataset_val["passage"], truncation=False, padding=False)
max_token_len_train = max([len(sequence) for sequence in train_tokens['input_ids']])
max_token_len_val = max([len(sequence) for sequence in val_tokens['input_ids']])
print(max_token_len_train)
print(max_token_len_val)

Token indices sequence length is longer than the specified maximum sequence length for this model (904 > 512). Running this sequence through the model will result in indexing errors


904
1055


In [ ]:
train_tokens = tokenizer(dataset_train["question"] + [' [SEP] '] + dataset_train["passage"], truncation=False, padding=False)
val_tokens = tokenizer(dataset_val["question"] + [' [SEP] '] + dataset_val["passage"], truncation=False, padding=False)
max_token_len_train = max([len(sequence) for sequence in train_tokens['input_ids']])
max_token_len_val = max([len(sequence) for sequence in val_tokens['input_ids']])
print(max_token_len_train)
print(max_token_len_val)

893
1044


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class BinaryClassificationIPairDataset(Dataset):
  def __init__(self, input_questions, input_passages, targets, tokenizer, summarizer, max_length):
    self.input_questions = input_questions
    self.input_passages = input_passages
    self.targets = targets
    self.tokenizer = tokenizer
    self.summarizer = summarizer
    self.max_length = max_length

  def __len__(self):
    return len(self.input_questions)

  def __getitem__(self, idx):
    
    #input_pair_str = self.input_questions[idx] + ' [SEP] ' + self.input_passages[idx]

    target = self.targets[idx]

    #print(self.input_passages[idx])
    #summarized_passage = self.summarizer(self.input_passages[idx], max_length=80, min_length=30, do_sample=False)[0]["summary_text"]
    #print(summarized_passage)

    encoding = self.tokenizer(
      #input_pair_str,
      text = self.input_questions[idx],
      #text_pair = summarized_passage,
      text_pair = self.input_passages[idx],
      add_special_tokens=True,
      max_length=self.max_length,
      padding='max_length',
      truncation="longest_first",
      return_attention_mask=True,
      return_token_type_ids=True,
      return_tensors='pt'
    )

    return {
      'input_ids': encoding['input_ids'].flatten(),
      'token_type_ids': encoding['token_type_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'labels': torch.tensor([target], dtype=torch.long)
    }

In [ ]:
train_input_questions = dataset_train["question"]
train_input_passages = dataset_train["passage"]
train_targets = dataset_train["label"]

val_input_questions = dataset_val["question"]
val_input_passages = dataset_val["passage"]
val_targets = dataset_val["label"]

bc_dataset_train = BinaryClassificationIPairDataset(train_input_questions, train_input_passages, train_targets, base_tokenizer, summarizer, 512)
bc_dataset_val = BinaryClassificationIPairDataset(val_input_questions, val_input_passages, val_targets, base_tokenizer, summarizer, 512)

In [ ]:
bc_dataset_train[0]

{'input_ids': tensor([    0,  5016, 10209,   260,     8,  9724,  5158,  7566,  1994,     5,
           276,  2777,     2,     2, 43020,   811,  2777,   480, 27775, 48229,
         49066,   642, 35423,    48, 43621, 16948,   338, 38155, 10659, 35423,
            27,   282,     6,   111, 38155,   862, 35423,    27,   282,    73,
           238,    67,   684,    30,    63,   253, 41433,   274,  2726,   118,
            36, 26068, 10172, 29438, 29434, 40637, 44148, 14285,   856, 18195,
          4926,   118,    36,   506, 35423, 10659, 43621, 16948, 35423,  4726,
         49066, 11000, 43621, 16948,    43,    36,  4161, 46934,    16,    65,
             9,     5,  2027,  5051, 11991,   624,     5, 27617,    12, 21336,
           811,  6084,     9,     5, 27617,    12, 17108,  2777,   284,     4,
            85,    16,  4212,  5826,    11,  1603,     6,  4035,    36, 27257,
         11416,   684,    25,   211,  1512,   187, 23102,   238,     8, 17335,
           967,  7566,    36, 27257, 11

In [ ]:
dataloader_train = DataLoader(bc_dataset_train, batch_size=12, shuffle=True)
dataloader_val = DataLoader(bc_dataset_val, batch_size=12, shuffle=True)

In [ ]:
for data in dataloader_train:
  print(data)
  break
for data in dataloader_val:
  print(data)
  break

{'input_ids': tensor([[    0,   354, 10267,  ...,     1,     1,     1],
        [    0,  7333,     5,  ...,     1,     1,     1],
        [    0,   354,    89,  ...,     1,     1,     1],
        ...,
        [    0,   354,   117,  ...,     1,     1,     1],
        [    0, 26692,  6325,  ...,     1,     1,     1],
        [    0,   354,    41,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [

# Train Model

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

args = TrainingArguments(
  output_dir="boolq_model",
  per_device_train_batch_size=12,
  per_device_eval_batch_size=12,
  evaluation_strategy="steps",
  eval_steps=20,
  logging_steps=20,
  gradient_accumulation_steps=16,
  num_train_epochs=2,
  #learning_rate=5e-2
)

# Lote de tamanho 12.
# Treino durante 2 épocas.

trainer = Trainer(
  model=base_model,
  tokenizer=tokenizer,
  data_collator=data_collator,
  args=args,
  train_dataset=bc_dataset_train,
  eval_dataset=bc_dataset_val,
  compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=98, training_loss=0.6270605106743015, metrics={'train_runtime': 2064.5334, 'train_samples_per_second': 9.132, 'train_steps_per_second': 0.047, 'total_flos': 4949382062376960.0, 'train_loss': 0.6270605106743015, 'epoch': 1.99})

In [ ]:
trainer.evaluate(bc_dataset_val)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

{'eval_loss': 0.5649130940437317,
 'eval_accuracy': 0.7045871559633028,
 'eval_f1': 0.764505119453925,
 'eval_runtime': 104.6845,
 'eval_samples_per_second': 31.237,
 'eval_steps_per_second': 2.608,
 'epoch': 1.99}

In [ ]:
trainer.save_model("boolq_model")

# Import and use Model

In [ ]:
config = RobertaConfig.from_pretrained(
    'boolq_model',
    num_labels=2,
)
model = RobertaForSequenceClassification.from_pretrained('boolq_model', config=config)

In [ ]:
import torch
def pre_process(passage, question):
  encoding = tokenizer(
    #input_pair_str,
    text = question,
    text_pair = passage,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation="longest_first",
    return_attention_mask=True,
    return_token_type_ids=True,
    return_tensors='pt'
  )

  return {
    'input_ids': encoding['input_ids'],
    'token_type_ids': encoding['token_type_ids'],
    'attention_mask': encoding['attention_mask']
  }


def test(passage, question):
  #target = pre_process(summarizer(passage, max_length=350, min_length=30, do_sample=False), question)
  target = pre_process(passage, question)
  
  return torch.argmax(model(
      input_ids=target["input_ids"],
      token_type_ids=target["token_type_ids"],
      attention_mask=target["attention_mask"]
    ).logits)


p1 = dataset_train["passage"][0]
q1 = dataset_train["question"][0]

print(p1)
print(q1)
print(test(p1, q1))


Persian language -- Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.
do iran and afghanistan speak the same language
tensor(1)
